In [1]:
# imports
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

# these three lines swap the stdlib sqlite3 lib with the pysqlite3 package
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
from langchain_chroma import Chroma

import os

In [ ]:
# initial setup
os.environ["OPENAI_API_KEY"] = ""
# Load dos modelos (Embedding model da OpenAI e a LLM em si)
embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(model_name= "gpt-3.5-turbo", max_tokens= 200)

In [3]:
# load pdf, split and create splitters
pdf_link = "os-sertoes.pdf"
loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

child_splitter = RecursiveCharacterTextSplitter(chunk_size=200) # small chunks
# large chunks
parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap = 200,
    length_function = len,
    add_start_index = True)

In [4]:
# create in memory store for parent chunks, store small chunks in the vector db
store = InMemoryStore()
vectorstore = Chroma(embedding_function=embeddings_model, persist_directory="child_vector_db")

In [5]:
# persist documents to the docstore (parent db)
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter)

parent_document_retriever.add_documents(pages, ids=None)

In [9]:
def ask(question):
    TEMPLATE = """
        Você é um especialista nas obras de Euclides da cunha, em especial, na obra "Os sertões". 
        Com base no contexto, responda a pergunta indicada.
        Contexto: {context}
        Pergunta: {question}
    """
    rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)
    setup_retrieval = RunnableParallel({"question": RunnablePassthrough(), "context": parent_document_retriever})
    output_parser = StrOutputParser()
    parent_chain_retrieval = setup_retrieval | rag_prompt | llm | output_parser
    response = parent_chain_retrieval.invoke(question)
    return response

In [10]:
response = ask("1. Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?")
response

'Euclides da Cunha descreve o ambiente natural do sertão nordestino como uma região árida e hostil, marcada por uma vegetação escassa e um clima extremamente seco. Ele destaca que essa paisagem inóspita exerce uma forte influência sobre a vida dos habitantes locais, tornando a sobrevivência uma tarefa árdua e desafiadora. Além disso, Euclides ressalta que as condições geográficas e climáticas do sertão contribuem para moldar a cultura, as tradições e o modo de vida das pessoas que habitam essa região, influenciando suas práticas cotidianas e suas relações sociais.'

In [12]:
response2 = ask("2. Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?")
response2

'Euclides da Cunha descreve a população sertaneja como resistente, adaptada às duras condições do ambiente árido e inóspito do sertão. Ele destaca a força física e psicológica dessas pessoas, que enfrentam as agruras do clima, a escassez de recursos e a violência do meio ambiente. O autor relaciona essas características com o ambiente em que vivem ao mostrar como a geografia e o clima do sertão moldam a vida e o comportamento dessas pessoas, influenciando sua cultura, suas tradições e sua forma de sobrevivência. Euclides da Cunha ressalta a importância de compreender a mesologia do local para entender a realidade da população sertaneja e sua relação com o meio ambiente hostil em que estão inseridos.'

In [13]:
response3 = ask("3. Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?")
response3

'Segundo Euclides da Cunha, o contexto histórico e político que levou à Guerra de Canudos foi marcado por uma série de expedições militares enviadas pelo governo para destruir a comunidade de Canudos, sob a alegação de que representava uma reação monárquica para solapar as instituições republicanas. A presença do beato Antônio Conselheiro e a formação da comunidade de Belo Monte às margens do Rio Vaza-Barris também foram fatores que contribuíram para o conflito. A divisão da mentalidade intelectual da época entre monarquistas e republicanos também influenciou o desenrolar dos eventos que culminaram na Guerra de Canudos.'

In [14]:
response4 = ask("4. Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?")
response4

'Euclides da Cunha descreve Antônio Conselheiro como um líder carismático e messiânico, que conseguiu reunir seguidores em torno de sua figura e de suas ideias. Ele é retratado como um homem que despertava devoção e fervor religioso em seus seguidores, sendo capaz de mobilizá-los em defesa de suas crenças e da comunidade de Canudos. Antônio Conselheiro é apresentado como o principal líder do movimento em Canudos, sendo visto como uma figura desafiadora e enigmática para as autoridades militares e políticas da época.'

In [15]:
response5 = ask('5. Quais são os principais aspectos da crítica social e política presentes em "Os Sertões"? Como esses aspectos refletem a visão do autor sobre o Brasil da época?')
response5

'Os principais aspectos da crítica social e política presentes em "Os Sertões" estão relacionados à desigualdade social, à violência, à injustiça e à marginalização dos sertanejos. Euclides da Cunha retrata a realidade geográfica e humana do sertão, destacando a luta do povo sertanejo contra as condições adversas e a opressão do governo e das forças militares.\n\nEsses aspectos refletem a visão do autor sobre o Brasil da época, mostrando a sua preocupação com as questões sociais e políticas do país. Euclides da Cunha critica a falta de desenvolvimento e de atenção do governo para com o sertão, evidenciando a marginalização e o abandono dessa região. Além disso, ele denuncia a violência e a brutalidade das for'